In [15]:
import pickle,os
import os.path as osp
import re
import nltk
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer=WordNetLemmatizer()
from ohbm.api import Api
import ohbm

In [3]:
access_token="0C78129639B37234B95DBC94DA64F74A"
url='https://ww5.aievolution.com/hbm1701'
api = Api(access_token,url)


Welcome, OHBM Trainer!


In [5]:
abstracts = api.Abstracts.getAbstracts()
#abstracts = api.Abstracts.getAbstracts(abstractTypeID='3',acceptedFlag='99')

Found 2533 abstracts!


In [11]:
ohbm.__file__

'/home/jb/code/ohbm/ohbm/__init__.py'

In [26]:
pckg_dir = osp.dirname(osp.dirname(ohbm.__file__))
data_dir = osp.join(pckg_dir, 'abstracts_data')
if not osp.isdir(data_dir): os.mkdir(data_dir)
pickle.dump(abstracts, open(osp.join(data_dir, 'abstracts.pkl'),'wb'))

In [21]:
a = abstracts[0]

In [29]:
a.keys()

dict_keys(['@id', 'materialsMethods', 'submissionNumber', 'endsOn', 'conclusion', 'abstractNumber', 'documents', 'events', '@updatedon', 'figures', 'speakers', 'shortTitle', 'abstractType', 'authors', 'estimatedAttendance', 'reference', 'category', 'embargoEndsOn', 'institution', 'status', 'categories', 'completedFlag', 'latebreakingFlag', 'startsOn', 'previewurl', 'purpose', 'estimatedCredits', 'results', 'approvedCredits', 'title', 'keywords', '@insertedon', 'acceptedFlag', 'planner', 'featuredFlag'])